# **Sistema basado en contenido**

In [ ]:
#Importacion de las librerias
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm 

In [ ]:
# Constante con el nombre del dataset
PATH = 'data.csv'

# Importar datos

In [ ]:
#Importacion y lectura del dataset
df = pd.read_csv(PATH)
df.shape

(100000, 10)

In [ ]:
#Lectura de los cinco primeros registros del dataset.
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,655,52,4,11482,300,4,8,2012,94,7
1,2713,90,3,6479,469,1,8,2012,33,5
2,409,17,2,25472,435,1,12,2001,196,4
3,1150,234,10,23950,529,2,23,2019,79,2
4,2424,390,5,13046,395,2,20,2010,200,4


# Recomendación de libro

In [ ]:
def normalize(data):
    '''
    Esta función normalizará los datos de entrada para que estén entre 0 y 1
    
      parámetros:
          data (List) : La lista de valores que desea normalizar
    
      Devuelve:
          Los datos de entrada se normalizaron entre 0 y 1
    '''
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    return [x/max_val for x in data]

In [ ]:
# normalizar las num_pages, calificaciones, columnas de precios
df['num_pages_norm'] = normalize(df['num_pages'].values)
df['book_rating_norm'] = normalize(df['book_rating'].values)
df['book_price_norm'] = normalize(df['book_price'].values)

In [ ]:
def ohe(df, enc_col):
    '''
      Esta función codificará en caliente la columna especificada y la volverá a agregar.
      en el marco de datos de entrada
    
      parámetros:
          df (DataFrame) : El marco de datos al que desea que se anexen los resultados
          enc_col (String) : La columna que desea OHE
    
      Devuelve:
          Las columnas OHE agregadas al marco de datos de entrada
    '''
    
    ohe_df = pd.get_dummies(df[enc_col])
    ohe_df.reset_index(drop = True, inplace = True)
    return pd.concat([df, ohe_df], axis = 1)

In [ ]:
# OHE sobre publish_year y género
df = ohe(df = df, enc_col = 'publish_year')
df = ohe(df = df, enc_col = 'book_genre')
df = ohe(df = df, enc_col = 'text_lang')

In [ ]:
# eliminar columnas redundantes
cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
df.drop(columns = cols, inplace = True)
df.set_index('book_id', inplace = True)

In [ ]:
class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        Esta función calculará la similitud del coseno entre dos vectores
        '''
        return dot(v1,v2)/(norm(v1)*norm(v2))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): El dataframe
        song_id (cadena): Representa el nombre de la canción
        n_rec (int): cantidad de rec que el usuario desea
        """
        
        # calcular la similitud de entrada book_id vector w.r.t todos los demás vectores
        inputVec = self.df.loc[book_id].values
        self.df['sim']= self.df.apply(lambda x: self.cosine_sim(inputVec,x.values), axis=1)
        
        # devuelve los libros especificados por el usuario n superior
        return self.df.nlargest(columns='sim',n=n_rec)

In [ ]:
t = df.sample(1000).copy()
cbr = CBRecommend(df = t)

In [ ]:
# Mostrar los datos
cbr.df.head()

,author_id,reader_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,...,8,9,10,1,2,3,4,5,6,7
book_id,,,,,,,,,,,,,,,,,,,,,
2187,185,1838,1,0.424286,0.4,0.965,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
820,147,19479,9,0.927143,1.0,0.270,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
819,12,15720,17,0.445714,0.9,0.590,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1815,228,25454,39,0.727143,1.0,0.165,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1915,450,27295,10,0.281429,0.6,0.955,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
cbr.recommend(book_id = t.index[0], n_rec = 5)

,author_id,reader_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,...,9,10,1,2,3,4,5,6,7,sim
book_id,,,,,,,,,,,,,,,,,,,,,
2187,185,1838,1,0.424286,0.4,0.965,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1.000000
410,433,4513,25,0.805714,0.9,0.110,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0.999976
1087,109,1149,8,0.520000,0.7,0.400,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0.999962
2875,442,4104,31,0.312857,0.2,0.315,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0.999951
342,344,3428,36,0.235714,0.2,0.830,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0.999950
